In [1]:
import roboticstoolbox as rtb
import numpy as np
import cv2 as cv
import glob

In [2]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#prepare object points like (0,0,0) (1,0,0) (2,0,0)... (6,5,0)
objp = np.zeros((8*6,3),np.float32)
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)
objp=0.01*objp #square length is 1.0cm

In [3]:
#Arrays to store object points and image points from all the images.
objpoints = []#3d points in real world
imgpoints = []#2d points in image plane

images = glob.glob('*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    #Find the chess board corners
    ret,corners = cv.findChessboardCorners(gray, (8,6),None)
    #If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (5,5),(-1,-1),criteria) #offset 5pixels from the corner
        imgpoints.append(corners2)
    #Draw and display the corners,
    cv.drawChessboardCorners(img,(8,6),corners2,ret)
    cv.imshow('img',img)
    cv.waitKey(2000)

In [4]:
len(images)

31

In [5]:
cv.destroyAllWindows()

In [ ]:
imgpoints

In [6]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [7]:
mtx

array([[934.44192173,   0.        , 333.2247427 ],
       [  0.        , 939.51291465, 171.10048061],
       [  0.        ,   0.        ,   1.        ]])

In [8]:
ret

0.3976952792847574

In [9]:
dist

array([[-5.35651039e-01,  1.53009433e+00,  1.47216506e-04,
        -2.40415479e-03, -3.83108955e+00]])

In [10]:
#Undistort
img = cv.imread(images[0])
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

dst = cv.undistort(img,mtx,dist,None,newcameramtx)
x,y,w,h = roi
dst = dst[y:y+h,x:x+w]
cv.imwrite("calibrate_result.png",dst)

True

In [11]:
#Usage of rvecs, tvecs
def draw(img,corners, imgpts):
    corner = tuple(corners[0].ravel().astype(int))
    img = cv.line(img,corner, tuple(imgpts[0].ravel().astype(int)),(255,0,0),5)
    img = cv.line(img,corner, tuple(imgpts[1].ravel().astype(int)),(0,255,0),5)
    img = cv.line(img,corner, tuple(imgpts[2].ravel().astype(int)),(0,0,255),5)
    return img

In [12]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.01)
objp = np.zeros((8*6,3),np.float32)
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)

axis = np.float32([[3,0,0],[0,3,0],[0,0,-3]]).reshape(-1,3)

In [ ]:
img = cv.imread(images[5])
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret,corners = cv.findChessboardCorners(gray, (8,6),None)

if  ret==True:
    corners2 = cv.cornerSubPix(gray, corners, (11,11),(-1,-1),criteria)

#Find the rotation and translation vectors
ret,rvecs,tvecs = cv.solvePnP(objp,corners2,mtx,dist)

#project 3D points to image plane
imgpts, jac = cv.projectPoints(axis,rvecs,tvecs,mtx,dist)

img = draw(img,corners2,imgpts)
cv.imshow('img',img)
k = cv.waitKey(0)
if k==ord('s'):
    cv.imwrite(fname[:6]+'.png',img)

cv.destroyAllWindows()